##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [3]:
import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 0s 0us/step


Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [17]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-12.281989 ,  -5.9167233,  -2.2277982,  10.386333 , -23.52727  ,
         14.840562 , -14.43533  ,  -6.8010936,  -9.661474 ,  -4.7402973]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [22]:
tf.nn.softmax(predictions).numpy()

array([[1.6436312e-12, 9.5544084e-10, 3.8219440e-08, 1.1495603e-02,
        2.1480322e-17, 9.8850435e-01, 1.9081835e-13, 3.9457290e-10,
        2.2588069e-11, 3.0982688e-09]], dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [23]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [24]:
loss_fn(y_train[:1], predictions).numpy()

0.011562242

In [25]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [26]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0670 - accuracy: 0.9796
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0617 - accuracy: 0.9802
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0517 - accuracy: 0.9827
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0463 - accuracy: 0.9852
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0420 - accuracy: 0.9867


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [27]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0689 - accuracy: 0.9815


[0.06894039362668991, 0.9815000295639038]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [28]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [29]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[3.20999560e-10, 1.81712922e-10, 9.16215583e-08, 3.81231584e-05,
        8.36986120e-15, 1.54907587e-10, 6.20604015e-15, 9.99961495e-01,
        8.81579965e-08, 1.24798802e-07],
       [5.02512809e-10, 1.67066972e-07, 9.99999762e-01, 1.16439663e-10,
        6.05496972e-24, 5.75331294e-09, 7.43879980e-08, 4.84648099e-18,
        6.33297303e-11, 1.71770646e-20],
       [1.17149144e-08, 9.99906898e-01, 1.23487371e-05, 1.14135716e-07,
        1.16685129e-07, 3.08827062e-08, 6.46255444e-07, 7.83808864e-05,
        1.42828310e-06, 3.33014647e-08],
       [9.99962449e-01, 1.18257270e-12, 3.13433447e-05, 6.86767052e-08,
        4.25517011e-10, 9.59037720e-07, 8.28089128e-07, 1.09916833e-07,
        5.64314373e-10, 4.23035544e-06],
       [4.26105640e-09, 4.61145362e-15, 3.56255396e-06, 1.47661900e-11,
        9.99951959e-01, 6.21563634e-11, 5.53782868e-08, 1.21179667e-07,
        4.80901430e-10, 4.43802564e-05]], dtype=float32)>